# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\krist\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Read your api key environment variable
load_dotenv()
from newsapi import NewsApiClient
api_key = os.getenv("news_api")

In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [21]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_top_headlines(q='bitcoin',
                                            category='business',
                                            language='en',
                                            country='us')
#bitcoin_headlines

In [23]:
bitcoin_articles = newsapi.get_everything(q="bitcoin",  from_param='2020-12-20',
                                      to='2020-12-21',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
#bitcoin_articles

In [6]:
bitcoin_df = pd.DataFrame.from_dict(bitcoin_articles["articles"])

bitcoin_df.head()

,author,content,description,publishedAt,source,title,url,urlToImage
0,Marvellous Aham-adi,Looking for the best new CMS plugins to take y...,Looking for the best new CMS plugins to take y...,2020-12-21T10:45:17Z,"{'id': None, 'name': 'Webdesignerdepot.com'}","Top New CMS Plugins, December 2020",https://www.webdesignerdepot.com/2020/12/top-n...,https://www.webdesignerdepot.com/cdn-origin/up...
1,"Roger Huang, Contributor, \n Roger Huang, Cont...",Safe Deposit. Symbol of cryptocurrency safety....,Quantum computing isn't going to suddenly end ...,2020-12-21T20:32:18Z,"{'id': None, 'name': 'Forbes'}",Here’s Why Quantum Computing Will Not Break Cr...,https://www.forbes.com/sites/rogerhuang/2020/1...,https://thumbor.forbes.com/thumbor/fit-in/1200...
2,"Billy Bambrough, Contributor, \n Billy Bambrou...",Tesla \r\n TSLA\r\n chief executive Elon Musk ...,"Musk, the billionaire serial entrepreneur who'...",2020-12-20T21:30:00Z,"{'id': None, 'name': 'Forbes'}",Elon Musk Sparks Wild Speculation Tesla Could ...,https://www.forbes.com/sites/billybambrough/20...,https://thumbor.forbes.com/thumbor/fit-in/1200...
3,Danny Nelson,MicroStrategy has invested all of the proceeds...,MicroStrategy is transforming itself into corp...,2020-12-21T13:26:43Z,"{'id': None, 'name': 'CoinDesk'}",MicroStrategy Splurges Another $650M in Latest...,https://www.coindesk.com/microstrategy-buys-bi...,https://static.coindesk.com/wp-content/uploads...
4,"Ron Shevlin, Senior Contributor, \n Ron Shevli...",(Photo by Nicolas Economou/NurPhoto via Getty ...,It’s not easy getting good rewards on your che...,2020-12-20T13:00:00Z,"{'id': None, 'name': 'Forbes'}",How To Earn Bitcoin Rewards On Your Stimulus C...,https://www.forbes.com/sites/ronshevlin/2020/1...,https://thumbor.forbes.com/thumbor/fit-in/1200...


In [123]:
bit_df = bitcoin_df.drop(columns = {"author", "description", "publishedAt","source","url", "urlToImage"})
bit_df.head()

,content,title
0,Looking for the best new CMS plugins to take y...,"Top New CMS Plugins, December 2020"
1,Safe Deposit. Symbol of cryptocurrency safety....,Here’s Why Quantum Computing Will Not Break Cr...
2,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...
3,MicroStrategy has invested all of the proceeds...,MicroStrategy Splurges Another $650M in Latest...
4,(Photo by Nicolas Economou/NurPhoto via Getty ...,How To Earn Bitcoin Rewards On Your Stimulus C...


In [124]:
# Fetch the Ethereum news articles
ethereum_articles = newsapi.get_everything(q="ethereum",  from_param='2020-12-20',
                                      to='2020-12-21',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
ethereum_df = pd.DataFrame.from_dict(ethereum_articles["articles"])

e_df = ethereum_df.drop(columns = {"author", "description", "publishedAt","source","url", "urlToImage"})
e_df.head()

,content,title
0,Ethereum pared a small portion of its recent g...,Ethereum Weekly Setup Sees Price Near $980 by ...
1,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...
2,"Bitcoin has surged higher again, soaring to fr...",Coinbase CEO Brian Armstrong Issues Serious Wa...
3,"After Bitcoin, the top-ranked altcoin, Ethereu...",Prelude To Altcoin Season: Ethereum Active Add...
4,Crypto Stripe Flexa Network has completed an a...,"Robot Ventures, Others Back ‘Crypto Stripe’ Fl..."


In [9]:
# Create the Bitcoin sentiment scores DataFrame
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [11]:
title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    "title_sent": [],
}
text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text and the title
for index, row in bit_df.iterrows():
    try:
           # Sentiment scoring with VADER
        title_sentiment = analyzer.polarity_scores(row["title"])
        title_sent["title_compound"].append(title_sentiment["compound"])
        title_sent["title_pos"].append(title_sentiment["pos"])
        title_sent["title_neu"].append(title_sentiment["neu"])
        title_sent["title_neg"].append(title_sentiment["neg"])
        title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))
        
        text_sentiment = analyzer.polarity_scores(row["content"])
        text_sent["text_compound"].append(text_sentiment["compound"])
        text_sent["text_pos"].append(text_sentiment["pos"])
        text_sent["text_neu"].append(text_sentiment["neu"])
        text_sent["text_neg"].append(text_sentiment["neg"])
        text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
title_sentiment_df = pd.DataFrame(title_sent)
text_sentiment_df = pd.DataFrame(text_sent)
bitcoin_df_sentiment_title = bit_df.join(title_sentiment_df)


bitcoin_df_sentiment_title.head() 

,content,title,title_compound,title_pos,title_neu,title_neg,title_sent
0,Looking for the best new CMS plugins to take y...,"Top New CMS Plugins, December 2020",0.2023,0.265,0.735,0.0,1
1,Safe Deposit. Symbol of cryptocurrency safety....,Here’s Why Quantum Computing Will Not Break Cr...,0.0000,0.000,1.000,0.0,0
2,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...,0.0000,0.000,1.000,0.0,0
3,MicroStrategy has invested all of the proceeds...,MicroStrategy Splurges Another $650M in Latest...,0.0000,0.000,1.000,0.0,0
4,(Photo by Nicolas Economou/NurPhoto via Getty ...,How To Earn Bitcoin Rewards On Your Stimulus C...,0.4767,0.279,0.721,0.0,1


In [12]:
bitcoin_df_sentiment_text = bit_df.join(text_sentiment_df)
bitcoin_df_sentiment_text.head()

,content,title,text_compound,text_pos,text_neu,text_neg,text_sent
0,Looking for the best new CMS plugins to take y...,"Top New CMS Plugins, December 2020",0.8860,0.253,0.703,0.044,1.0
1,Safe Deposit. Symbol of cryptocurrency safety....,Here’s Why Quantum Computing Will Not Break Cr...,0.2500,0.157,0.714,0.129,1.0
2,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...,0.0000,0.000,1.000,0.000,0.0
3,MicroStrategy has invested all of the proceeds...,MicroStrategy Splurges Another $650M in Latest...,-0.3612,0.000,0.928,0.072,-1.0
4,(Photo by Nicolas Economou/NurPhoto via Getty ...,How To Earn Bitcoin Rewards On Your Stimulus C...,0.0000,0.000,1.000,0.000,0.0


In [13]:
# Create the ethereum sentiment scores DataFrame
e_title_sent = {
    "title_compound": [],
    "title_pos": [],
    "title_neu": [],
    "title_neg": [],
    "title_sent": [],
}
e_text_sent = {
    "text_compound": [],
    "text_pos": [],
    "text_neu": [],
    "text_neg": [],
    "text_sent": [],
}

# Get sentiment for the text and the title
for index, row in e_df.iterrows():
    try:
        # Sentiment scoring with VADER
        e_title_sentiment = analyzer.polarity_scores(row["title"])
        e_title_sent["title_compound"].append(title_sentiment["compound"])
        e_title_sent["title_pos"].append(title_sentiment["pos"])
        e_title_sent["title_neu"].append(title_sentiment["neu"])
        e_title_sent["title_neg"].append(title_sentiment["neg"])
        e_title_sent["title_sent"].append(get_sentiment(title_sentiment["compound"]))
        
        e_text_sentiment = analyzer.polarity_scores(row["content"])
        e_text_sent["text_compound"].append(text_sentiment["compound"])
        e_text_sent["text_pos"].append(text_sentiment["pos"])
        e_text_sent["text_neu"].append(text_sentiment["neu"])
        e_text_sent["text_neg"].append(text_sentiment["neg"])
        e_text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
e_title_sentiment_df = pd.DataFrame(e_title_sent)
e_text_sentiment_df = pd.DataFrame(e_text_sent)
ethereum_df_sentiment = e_df.join(e_title_sentiment_df)

ethereum_df_sentiment.head()

,content,title,title_compound,title_pos,title_neu,title_neg,title_sent
0,Ethereum pared a small portion of its recent g...,Ethereum Weekly Setup Sees Price Near $980 by ...,-0.5719,0.0,0.654,0.346,-1
1,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...,-0.5719,0.0,0.654,0.346,-1
2,"Bitcoin has surged higher again, soaring to fr...",Coinbase CEO Brian Armstrong Issues Serious Wa...,-0.5719,0.0,0.654,0.346,-1
3,"After Bitcoin, the top-ranked altcoin, Ethereu...",Prelude To Altcoin Season: Ethereum Active Add...,-0.5719,0.0,0.654,0.346,-1
4,Crypto Stripe Flexa Network has completed an a...,"Robot Ventures, Others Back ‘Crypto Stripe’ Fl...",-0.5719,0.0,0.654,0.346,-1


In [14]:
ethereum_df_sentiment_text = e_df.join(e_text_sentiment_df)
ethereum_df_sentiment_text.head()

,content,title,text_compound,text_pos,text_neu,text_neg,text_sent
0,Ethereum pared a small portion of its recent g...,Ethereum Weekly Setup Sees Price Near $980 by ...,0.4019,0.083,0.917,0.0,1
1,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...,0.4019,0.083,0.917,0.0,1
2,"Bitcoin has surged higher again, soaring to fr...",Coinbase CEO Brian Armstrong Issues Serious Wa...,0.4019,0.083,0.917,0.0,1
3,"After Bitcoin, the top-ranked altcoin, Ethereu...",Prelude To Altcoin Season: Ethereum Active Add...,0.4019,0.083,0.917,0.0,1
4,Crypto Stripe Flexa Network has completed an a...,"Robot Ventures, Others Back ‘Crypto Stripe’ Fl...",0.4019,0.083,0.917,0.0,1


In [15]:
# Describe the Bitcoin Sentiment
bitcoin_df_sentiment_title.describe()

,title_compound,title_pos,title_neu,title_neg,title_sent
count,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.107050,0.105600,0.845250,0.049150,0.200000
std,0.324743,0.166822,0.178217,0.104516,0.695852
min,-0.571900,0.000000,0.471000,0.000000,-1.000000
25%,0.000000,0.000000,0.701000,0.000000,0.000000
50%,0.000000,0.000000,0.928500,0.000000,0.000000
75%,0.348750,0.210000,1.000000,0.022250,1.000000
max,0.822500,0.529000,1.000000,0.346000,1.000000


In [16]:
# Describe the Ethereum Sentiment
ethereum_df_sentiment.describe()

,title_compound,title_pos,title_neu,title_neg,title_sent
count,2.000000e+01,20.0,20.000,2.000000e+01,20.0
mean,-5.719000e-01,0.0,0.654,3.460000e-01,-1.0
std,1.139065e-16,0.0,0.000,5.695324e-17,0.0
min,-5.719000e-01,0.0,0.654,3.460000e-01,-1.0
25%,-5.719000e-01,0.0,0.654,3.460000e-01,-1.0
50%,-5.719000e-01,0.0,0.654,3.460000e-01,-1.0
75%,-5.719000e-01,0.0,0.654,3.460000e-01,-1.0
max,-5.719000e-01,0.0,0.654,3.460000e-01,-1.0


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin

Q: Which coin had the highest compound score?

A: Bitcoin 

Q. Which coin had the highest positive score?

A: Bitcoin - Ethereum had a very negative sentiment, nothing was positive

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [179]:
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [180]:
# Expand the default stopwords list if necessary
# Code to download corpora
import nltk
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
sw = set(stopwords.words('english'))


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\krist\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\krist\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [219]:
bit_df["content"] = bit_df["content"].fillna('').apply(str)
#.values.tolist()
bit_text2 = bit_df["content"][2]

In [242]:
bit_df["tokens"] = bit_df.apply(lambda row: nltk.word_tokenize(row['content']), axis=1)
token_bit = bit_df["tokens"]
token_bit.head()

0    [Looking, for, the, best, new, CMS, plugins, t...
1    [Safe, Deposit, ., Symbol, of, cryptocurrency,...
2    [Tesla, TSLA, chief, executive, Elon, Musk, ha...
3    [MicroStrategy, has, invested, all, of, the, p...
4    [(, Photo, by, Nicolas, Economou/NurPhoto, via...
Name: tokens, dtype: object

In [236]:
# Remove the punctuation
regex = re.compile("[^a-zA-Z ]")
re_clean = regex.sub('',bit_text2 )
#isolate article words
re_clean

'Tesla  TSLA chief executive Elon Musk has sparked speculation his electric car company could add bitcoin to its balance sheet during a public exchange on TwitterMusk the billionaire serial en  chars'

In [237]:
    # Lemmatize Words into root words
words = word_tokenize(re_clean)
lem = [lemmatizer.lemmatize(word) for word in words]
lem

['Tesla',
 'TSLA',
 'chief',
 'executive',
 'Elon',
 'Musk',
 'ha',
 'sparked',
 'speculation',
 'his',
 'electric',
 'car',
 'company',
 'could',
 'add',
 'bitcoin',
 'to',
 'it',
 'balance',
 'sheet',
 'during',
 'a',
 'public',
 'exchange',
 'on',
 'TwitterMusk',
 'the',
 'billionaire',
 'serial',
 'en',
 'char']

In [238]:
lower = [word.lower() for word in words if word.lower() not in sw]
lower

['tesla',
 'tsla',
 'chief',
 'executive',
 'elon',
 'musk',
 'sparked',
 'speculation',
 'electric',
 'car',
 'company',
 'could',
 'add',
 'bitcoin',
 'balance',
 'sheet',
 'public',
 'exchange',
 'twittermusk',
 'billionaire',
 'serial',
 'en',
 'chars']

In [239]:
# Complete the tokenizer function
lemmatizer = WordNetLemmatizer()
bit_text = bit_df["content"][3]

def tokenizer(text):
    """Tokenizes text."""
    
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    
    words = word_tokenize(re_clean)
    
    
    #e_text = e_df["content"].values.tolist()
   

  
    # Remove the stop words
    sentence_tokenized = [sent_tokenize(word) for word in text]
    
    # Lemmatize Words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    
    # Convert the words to lowercase
    lower = [word.lower() for word in words if word.lower() not in sw]
    return lower
print(set(tokenizer(bit_text)))

{'tweet', 'million', 'debt', 'michael', 'latest', 'made', 'monday', 'announced', 'saylor', 'bitcoinulliceo', 'chars', 'invested', 'proceeds', 'purchase', 'issuance', 'microstrategy'}


In [250]:
# Create a new tokens column for bitcoin
#bitcoin_df_sentiment_text['Token'] = np.nan
#bit_tokens = []
#for text in bitcoin_tokens:
    #raw_stories.append(reuters.raw(id))
    #ids.append(id)
#for index, row in amd_df.iterrows()     
#bitcoin_df_sentiment_text["tokens"]
#for i in bitcoin_df_sentiment_text['content'][0],[i]:
     #tokens = set(tokenizer(text))
bc_df_tokens = bitcoin_df_sentiment_text.join(token_bit)
bc_df_tokens.drop(columns = 'Token')
bc_df_tokens.head()
#bitcoin_tokens    
#bit_tokens = bitcoin_df_sentiment_text({"tokens":bitcoin_tokens})
#bit_tokens.head()
# YOUR CODE HERE!

,content,title,text_compound,text_pos,text_neu,text_neg,text_sent,Token,tokens
0,Looking for the best new CMS plugins to take y...,"Top New CMS Plugins, December 2020",0.8860,0.253,0.703,0.044,1.0,NaN,"[Looking, for, the, best, new, CMS, plugins, t..."
1,Safe Deposit. Symbol of cryptocurrency safety....,Here’s Why Quantum Computing Will Not Break Cr...,0.2500,0.157,0.714,0.129,1.0,NaN,"[Safe, Deposit, ., Symbol, of, cryptocurrency,..."
2,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...,0.0000,0.000,1.000,0.000,0.0,NaN,"[Tesla, TSLA, chief, executive, Elon, Musk, ha..."
3,MicroStrategy has invested all of the proceeds...,MicroStrategy Splurges Another $650M in Latest...,-0.3612,0.000,0.928,0.072,-1.0,NaN,"[MicroStrategy, has, invested, all, of, the, p..."
4,(Photo by Nicolas Economou/NurPhoto via Getty ...,How To Earn Bitcoin Rewards On Your Stimulus C...,0.0000,0.000,1.000,0.000,0.0,NaN,"[(, Photo, by, Nicolas, Economou/NurPhoto, via..."


In [251]:
e_df["tokens"] = e_df.apply(lambda row: nltk.word_tokenize(row['content']), axis=1)
e_token = e_df["tokens"]
e_token.head()

0    [Ethereum, pared, a, small, portion, of, its, ...
1    [Tesla, TSLA, chief, executive, Elon, Musk, ha...
2    [Bitcoin, has, surged, higher, again, ,, soari...
3    [After, Bitcoin, ,, the, top-ranked, altcoin, ...
4    [Crypto, Stripe, Flexa, Network, has, complete...
Name: tokens, dtype: object

In [252]:
# Create a new tokens column for ethereum
e_df_tokens = ethereum_df_sentiment_text.join(e_token)
e_df_tokens.head()

,content,title,text_compound,text_pos,text_neu,text_neg,text_sent,tokens
0,Ethereum pared a small portion of its recent g...,Ethereum Weekly Setup Sees Price Near $980 by ...,0.4019,0.083,0.917,0.0,1,"[Ethereum, pared, a, small, portion, of, its, ..."
1,Tesla \r\n TSLA\r\n chief executive Elon Musk ...,Elon Musk Sparks Wild Speculation Tesla Could ...,0.4019,0.083,0.917,0.0,1,"[Tesla, TSLA, chief, executive, Elon, Musk, ha..."
2,"Bitcoin has surged higher again, soaring to fr...",Coinbase CEO Brian Armstrong Issues Serious Wa...,0.4019,0.083,0.917,0.0,1,"[Bitcoin, has, surged, higher, again, ,, soari..."
3,"After Bitcoin, the top-ranked altcoin, Ethereu...",Prelude To Altcoin Season: Ethereum Active Add...,0.4019,0.083,0.917,0.0,1,"[After, Bitcoin, ,, the, top-ranked, altcoin, ..."
4,Crypto Stripe Flexa Network has completed an a...,"Robot Ventures, Others Back ‘Crypto Stripe’ Fl...",0.4019,0.083,0.917,0.0,1,"[Crypto, Stripe, Flexa, Network, has, complete..."


---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [253]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Get the top 10 words for Ethereum
# YOUR CODE HERE!

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

## Ethereum NER

In [ ]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!